In [92]:
import pandas as pd
import importlib
import Forage
from Agent import HoneyBee
from itertools import product

# ——————————————————————————————————————————
# 1) Configuración de la simulación
# ——————————————————————————————————————————
Time_sim        = 5000
resource_value  = 1

Nexplorer_range   = [20]
Nexploiter_range  = [10]
dist_colony_range = [120]
speed_x_range     = [7]
speed_y_range     = [7]
n_range           = [4]
d_x_range         = [1]
d_y_range         = [1]

hive_x, hive_y = HoneyBee.hiveX, HoneyBee.hiveY

importlib.reload(Forage)
from Forage import Forage

# ——————————————————————————————————————————
# 2) Ejecutar todas las simulaciones y almacenar en memoria
# ——————————————————————————————————————————
all_dfs = []

for (Nexplorer, Nexploiter,
     dist_colony,
     speed_x, speed_y,
     n, d_x, d_y) in product(
        Nexplorer_range, Nexploiter_range,
        dist_colony_range,
        speed_x_range, speed_y_range,
        n_range, d_x_range, d_y_range
    ):

    sim = Forage(
        Nexplorer=Nexplorer,
        Nexploiter=Nexploiter,
        T=Time_sim,
        SctPersistence=20,
        RecPersistence=25
    )
    target_zone = (hive_x + dist_colony, hive_y)
    sim.resourcesMatrixSpacing(
        n=n, d_x=d_x, d_y=d_y,
        resource_value=resource_value,
        start_x=target_zone[0] + 2,
        start_y=target_zone[1] - 1
    )
    for bee in sim.Explorers + sim.Exploiters:
        bee.target   = target_zone
        bee.capacity = 2

    sim.speed_x = speed_x
    sim.speed_y = speed_y
    sim.total_collected = 0

    rows = []
    for step in range(Time_sim):
        prev_len    = len(sim.visitLog)
        sim.update()
        new_events  = sim.visitLog[prev_len:]
        visitas_con = sum(1 for ev in new_events if ev[0] == 'visita_con_recurso')
        visitas_sin = sum(1 for ev in new_events if ev[0] == 'visita_sin_recurso')
        queue_wait  = sum(wait for (_, _, wait) in sim.flowerQueue)
        collected   = sim.total_collected

        rows.append({
            'd_x':         d_x,
            'd_y':         d_y,
            'speed_x':     speed_x,
            'speed_y':     speed_y,
            'dist_colony': dist_colony,
            'Nexpl':       Nexplorer,
            'Nexploit':    Nexploiter,
            'n':           n,
            'step':        step,
            'visitas_con': visitas_con,
            'visitas_sin': visitas_sin,
            'queue_wait':  queue_wait,
            'collected':   collected
        })

    df_sim_rows = pd.DataFrame(rows)
    all_dfs.append(df_sim_rows)

# ——————————————————————————————————————————
# 3) Concatenar todos los resultados en un DataFrame maestro
# ——————————————————————————————————————————
master_df = pd.concat(all_dfs, ignore_index=True)

# ——————————————————————————————————————————
# 4) Generar summary_df con las métricas agregadas
# ——————————————————————————————————————————
summary = {
    "d_x":                      master_df['d_x'].iloc[0],
    "d_y":                      master_df['d_y'].iloc[0],
    "speed_x":                  master_df['speed_x'].iloc[0],
    "speed_y":                  master_df['speed_y'].iloc[0],
    "dist_colony":              master_df['dist_colony'].iloc[0],
    "Nexpl":                    master_df['Nexpl'].iloc[0],
    "Nexploit":                 master_df['Nexploit'].iloc[0],
    "n":                        master_df['n'].iloc[0],
    "visitas_con_nectar":       master_df['visitas_con'].sum(),
    "visitas_sin_nectar":       master_df['visitas_sin'].sum(),
    "media_tiempo_queue":       master_df['queue_wait'].mean(),
    "total_nectar_recolectado": master_df['collected'].max()
}
summary_df = pd.DataFrame([summary])

# ——————————————————————————————————————————
# 5) Calcular eficiencia usando summary_df
# ——————————————————————————————————————————
# Cargar base histórica y parámetros
df_base = pd.read_csv('datos_resumen.csv').rename(columns={
    'Nexpl': 'nexpl', 'Nexploit': 'nexpt', 'dist_colony': 'dist_col'
})
w1, w2, w3, w4 = 0.3, 0.3, 0.2, 0.2
output_cols = ['visitas_con', 'nectar', 'visitas_sin', 't_refill_med']
min_vals = df_base[output_cols].min()
max_vals = df_base[output_cols].max()

def calcular_eficiencia_de_df(df_sum, df_base, min_vals, max_vals):
    fila = df_sum.iloc[0]
    c1 = (fila['visitas_con_nectar']       - min_vals['visitas_con']) / (max_vals['visitas_con']   - min_vals['visitas_con'])
    c2 = (fila['total_nectar_recolectado']  - min_vals['nectar'])      / (max_vals['nectar']        - min_vals['nectar'])
    c3 = (fila['visitas_sin_nectar']       - min_vals['visitas_sin']) / (max_vals['visitas_sin']   - min_vals['visitas_sin'])
    c4 = (fila['media_tiempo_queue']       - min_vals['t_refill_med'])/ (max_vals['t_refill_med']  - min_vals['t_refill_med'])
    E_raw_new = w1*c1 + w2*c2 - w3*c3 - w4*c4

    norm_base = (
        w1 * (df_base['visitas_con']   - min_vals['visitas_con']) / (max_vals['visitas_con']   - min_vals['visitas_con'])
      + w2 * (df_base['nectar']        - min_vals['nectar'])      / (max_vals['nectar']        - min_vals['nectar'])
      - w3 * (df_base['visitas_sin']   - min_vals['visitas_sin']) / (max_vals['visitas_sin']   - min_vals['visitas_sin'])
      - w4 * (df_base['t_refill_med']  - min_vals['t_refill_med'])/ (max_vals['t_refill_med']  - min_vals['t_refill_med'])
    )
    E_min, E_max = norm_base.min(), norm_base.max()
    return (E_raw_new - E_min) / (E_max - E_min)

# Aplicar y añadir eficiencia
eficiencia = calcular_eficiencia_de_df(summary_df, df_base, min_vals, max_vals)
summary_df['Eficiencia'] = eficiencia

# Ahora master_df y summary_df están disponibles en memoria para usar más adelante
summary_df


,d_x,d_y,speed_x,speed_y,dist_colony,Nexpl,Nexploit,n,visitas_con_nectar,visitas_sin_nectar,media_tiempo_queue,total_nectar_recolectado,Eficiencia
0,1,1,7,7,120,20,10,4,327,383,202.8,381,0.800091


In [114]:
import pandas as pd
import importlib
import Forage
from Agent import HoneyBee
from itertools import product
import numpy as np

# ——————————————————————————————————————————
# 1) Configuración de la simulación
# ——————————————————————————————————————————
Time_sim        = 5000
resource_value  = 1
k_replicas      = 20   # ← número de réplicas por combinación

# Rango de parámetros
Nexplorer_range   = [20]
Nexploiter_range  = [10]
dist_colony_range = [120]
speed_x_range     = [7]
speed_y_range     = [7]
n_range           = [4]
d_x_range         = [1]
d_y_range         = [1]

hive_x, hive_y = HoneyBee.hiveX, HoneyBee.hiveY

importlib.reload(Forage)
from Forage import Forage

# ——————————————————————————————————————————
# 2) Ejecutar simulaciones: recojo cada réplica en all_replicas
# ——————————————————————————————————————————
all_replicas = []

for (Nexplorer, Nexploiter,
     dist_colony,
     speed_x, speed_y,
     n, d_x, d_y) in product(
        Nexplorer_range, Nexploiter_range,
        dist_colony_range,
        speed_x_range, speed_y_range,
        n_range, d_x_range, d_y_range
    ):

    for rep in range(1, k_replicas+1):
        sim = Forage(
            Nexplorer=Nexplorer,
            Nexploiter=Nexploiter,
            T=Time_sim,
            SctPersistence=20,
            RecPersistence=25
        )
        target_zone = (hive_x + dist_colony, hive_y)
        sim.resourcesMatrixSpacing(
            n=n, d_x=d_x, d_y=d_y,
            resource_value=resource_value,
            start_x=target_zone[0] + 2,
            start_y=target_zone[1] - 1
        )
        for bee in sim.Explorers + sim.Exploiters:
            bee.target   = target_zone
            bee.capacity = 2

        sim.speed_x = speed_x
        sim.speed_y = speed_y
        sim.total_collected = 0

        visitas_con_total = 0
        visitas_sin_total = 0

        for step in range(Time_sim):
            prev_len    = len(sim.visitLog)
            sim.update()
            new_events  = sim.visitLog[prev_len:]
            visitas_con_total += sum(1 for ev in new_events if ev[0] == 'visita_con_recurso')
            visitas_sin_total += sum(1 for ev in new_events if ev[0] == 'visita_sin_recurso')

        nectar_total = sim.total_collected

        all_replicas.append({
            'replica':                 rep,
            'd_x':                      d_x,
            'd_y':                      d_y,
            'speed_x':                  speed_x,
            'speed_y':                  speed_y,
            'dist_colony':              dist_colony,
            'Nexpl':                    Nexplorer,
            'Nexploit':                 Nexploiter,
            'n':                        n,
            'visitas_con_nectar':       visitas_con_total,
            'visitas_sin_nectar':       visitas_sin_total,
            'total_nectar_recolectado': nectar_total
        })

# ——————————————————————————————————————————
# 3) Mostrar todos los resultados individuales (k réplicas)
# ——————————————————————————————————————————
all_replicas_df = pd.DataFrame(all_replicas)
print("=== Resultados de cada réplica ===")
print(all_replicas_df)

# ——————————————————————————————————————————
# 4) Promediar y generar summary_df
# ——————————————————————————————————————————
group_cols = ['d_x','d_y','speed_x','speed_y','dist_colony','Nexpl','Nexploit','n']
summary = all_replicas_df.groupby(group_cols).agg({
    'visitas_con_nectar':       'mean',
    'visitas_sin_nectar':       'mean',
    'total_nectar_recolectado': 'mean'
}).reset_index()

# renombramos para que coincida con los nombres anteriores
summary = summary.rename(columns={
    'visitas_con_nectar':       'visitas_con_nectar',
    'visitas_sin_nectar':       'visitas_sin_nectar',
    'total_nectar_recolectado': 'total_nectar_recolectado'
})

# ——————————————————————————————————————————
# 5) Calcular eficiencia
# ——————————————————————————————————————————
df_base = pd.read_csv('datos_resumen.csv').rename(columns={
    'Nexpl': 'nexpl', 'Nexploit': 'nexpt', 'dist_colony': 'dist_col'
})
w1, w2, w3 = 0.4, 0.4, 0.2
output_cols = ['visitas_con', 'nectar', 'visitas_sin']
min_vals = df_base[output_cols].min()
max_vals = df_base[output_cols].max()

def calcular_eficiencia_sin_tfill(fila):
    c1 = (fila['visitas_con_nectar']      - min_vals['visitas_con']) / (max_vals['visitas_con']   - min_vals['visitas_con'])
    c2 = (fila['total_nectar_recolectado'] - min_vals['nectar'])      / (max_vals['nectar']        - min_vals['nectar'])
    c3 = (fila['visitas_sin_nectar']      - min_vals['visitas_sin']) / (max_vals['visitas_sin']   - min_vals['visitas_sin'])
    E_raw = w1*c1 + w2*c2 - w3*c3

    norm_base = (
        w1 * (df_base['visitas_con']   - min_vals['visitas_con']) / (max_vals['visitas_con']   - min_vals['visitas_con'])
      + w2 * (df_base['nectar']        - min_vals['nectar'])      / (max_vals['nectar']        - min_vals['nectar'])
      - w3 * (df_base['visitas_sin']   - min_vals['visitas_sin']) / (max_vals['visitas_sin']   - min_vals['visitas_sin'])
    )
    E_min, E_max = norm_base.min(), norm_base.max()
    return (E_raw - E_min) / (E_max - E_min)

summary['Eficiencia'] = summary.apply(calcular_eficiencia_sin_tfill, axis=1)

# ——————————————————————————————————————————
# 6) Mostrar la tabla final de promedios + eficiencia
# ——————————————————————————————————————————
print("\n=== Tabla final (promedios + eficiencia) ===")
print(summary)


=== Resultados de cada réplica ===
    replica  d_x  d_y  speed_x  speed_y  dist_colony  Nexpl  Nexploit  n  \
0         1    1    1        7        7          120     20        10  4   
1         2    1    1        7        7          120     20        10  4   
2         3    1    1        7        7          120     20        10  4   
3         4    1    1        7        7          120     20        10  4   
4         5    1    1        7        7          120     20        10  4   
5         6    1    1        7        7          120     20        10  4   
6         7    1    1        7        7          120     20        10  4   
7         8    1    1        7        7          120     20        10  4   
8         9    1    1        7        7          120     20        10  4   
9        10    1    1        7        7          120     20        10  4   
10       11    1    1        7        7          120     20        10  4   
11       12    1    1        7        7          120 

In [115]:
import pandas as pd
import importlib
import Forage
from Agent import HoneyBee
from itertools import product
import numpy as np

# ——————————————————————————————————————————
# 1) Configuración de la simulación
# ——————————————————————————————————————————
Time_sim        = 5000
resource_value  = 1
k_replicas      = 20   # número de réplicas por combinación

# Rango de parámetros
Nexplorer_range   = [20]
Nexploiter_range  = [10]
dist_colony_range = [120]
speed_x_range     = [7]
speed_y_range     = [7]
n_range           = [4]
d_x_range         = [1]
d_y_range         = [1]

hive_x, hive_y = HoneyBee.hiveX, HoneyBee.hiveY

importlib.reload(Forage)
from Forage import Forage

# ——————————————————————————————————————————
# 2) Ejecutar simulaciones: recojo cada réplica en all_replicas
# ——————————————————————————————————————————
all_replicas = []

for (Nexplorer, Nexploiter,
     dist_colony,
     speed_x, speed_y,
     n, d_x, d_y) in product(
        Nexplorer_range, Nexploiter_range,
        dist_colony_range,
        speed_x_range, speed_y_range,
        n_range, d_x_range, d_y_range
    ):

    for rep in range(1, k_replicas+1):
        sim = Forage(
            Nexplorer=Nexplorer,
            Nexploiter=Nexploiter,
            T=Time_sim,
            SctPersistence=20,
            RecPersistence=25
        )
        target_zone = (hive_x + dist_colony, hive_y)
        sim.resourcesMatrixSpacing(
            n=n, d_x=d_x, d_y=d_y,
            resource_value=resource_value,
            start_x=target_zone[0] + 2,
            start_y=target_zone[1] - 1
        )
        for bee in sim.Explorers + sim.Exploiters:
            bee.target   = target_zone
            bee.capacity = 2

        sim.speed_x = speed_x
        sim.speed_y = speed_y
        sim.total_collected = 0

        visitas_con_total = 0
        visitas_sin_total = 0
        queue_wait_sum    = 0

        for step in range(Time_sim):
            prev_len    = len(sim.visitLog)
            sim.update()
            new_events  = sim.visitLog[prev_len:]
            visitas_con_total += sum(1 for ev in new_events if ev[0] == 'visita_con_recurso')
            visitas_sin_total += sum(1 for ev in new_events if ev[0] == 'visita_sin_recurso')
            # sumo la espera total en la cola este paso
            queue_wait_sum += sum(wait for (_, _, wait) in sim.flowerQueue)

        nectar_total = sim.total_collected
        # calculo promedio de tiempo de queue por paso
        t_medio_reposicion = queue_wait_sum / Time_sim

        all_replicas.append({
            'replica':                  rep,
            'd_x':                       d_x,
            'd_y':                       d_y,
            'speed_x':                   speed_x,
            'speed_y':                   speed_y,
            'dist_colony':               dist_colony,
            'Nexpl':                     Nexplorer,
            'Nexploit':                  Nexploiter,
            'n':                         n,
            'visitas_con_nectar':        visitas_con_total,
            'visitas_sin_nectar':        visitas_sin_total,
            'total_nectar_recolectado':  nectar_total,
            't_medio_reposicion':        t_medio_reposicion
        })

# ——————————————————————————————————————————
# 3) Mostrar todos los resultados individuales (k réplicas)
# ——————————————————————————————————————————
all_replicas_df = pd.DataFrame(all_replicas)
print("=== Resultados de cada réplica ===")
print(all_replicas_df)

# ——————————————————————————————————————————
# 4) Promediar y generar summary_df
# ——————————————————————————————————————————
group_cols = ['d_x','d_y','speed_x','speed_y','dist_colony','Nexpl','Nexploit','n']
summary = all_replicas_df.groupby(group_cols).agg({
    'visitas_con_nectar':       'mean',
    'visitas_sin_nectar':       'mean',
    'total_nectar_recolectado': 'mean',
    't_medio_reposicion':       'mean'
}).reset_index()

# ——————————————————————————————————————————
# 5) Calcular eficiencia con 4 métricas
# ——————————————————————————————————————————
df_base = pd.read_csv('datos_resumen.csv').rename(columns={
    'Nexpl': 'nexpl', 'Nexploit': 'nexpt', 'dist_colony': 'dist_col'
})
# pesos originales para 4 métricas (suman 1)
w1, w2, w3, w4 = 0.3, 0.3, 0.2, 0.2
output_cols = ['visitas_con', 'nectar', 'visitas_sin', 't_refill_med']
min_vals = df_base[output_cols].min()
max_vals = df_base[output_cols].max()

def calcular_eficiencia_con_tfill(fila):
    c1 = (fila['visitas_con_nectar']       - min_vals['visitas_con'])    / (max_vals['visitas_con']    - min_vals['visitas_con'])
    c2 = (fila['total_nectar_recolectado']  - min_vals['nectar'])         / (max_vals['nectar']         - min_vals['nectar'])
    c3 = (fila['visitas_sin_nectar']       - min_vals['visitas_sin'])    / (max_vals['visitas_sin']    - min_vals['visitas_sin'])
    c4 = (fila['t_medio_reposicion']       - min_vals['t_refill_med'])   / (max_vals['t_refill_med']   - min_vals['t_refill_med'])

    E_raw = w1*c1 + w2*c2 - w3*c3 - w4*c4

    norm_base = (
        w1 * (df_base['visitas_con']   - min_vals['visitas_con'])    / (max_vals['visitas_con']    - min_vals['visitas_con'])
      + w2 * (df_base['nectar']        - min_vals['nectar'])         / (max_vals['nectar']         - min_vals['nectar'])
      - w3 * (df_base['visitas_sin']   - min_vals['visitas_sin'])    / (max_vals['visitas_sin']    - min_vals['visitas_sin'])
      - w4 * (df_base['t_refill_med']  - min_vals['t_refill_med'])   / (max_vals['t_refill_med']   - min_vals['t_refill_med'])
    )
    E_min, E_max = norm_base.min(), norm_base.max()
    return (E_raw - E_min) / (E_max - E_min)

summary['Eficiencia'] = summary.apply(calcular_eficiencia_con_tfill, axis=1)

# ——————————————————————————————————————————
# 6) Mostrar la tabla final de promedios + eficiencia
# ——————————————————————————————————————————
print("\n=== Tabla final (promedios + eficiencia) ===")
print(summary)


=== Resultados de cada réplica ===
    replica  d_x  d_y  speed_x  speed_y  dist_colony  Nexpl  Nexploit  n  \
0         1    1    1        7        7          120     20        10  4   
1         2    1    1        7        7          120     20        10  4   
2         3    1    1        7        7          120     20        10  4   
3         4    1    1        7        7          120     20        10  4   
4         5    1    1        7        7          120     20        10  4   
5         6    1    1        7        7          120     20        10  4   
6         7    1    1        7        7          120     20        10  4   
7         8    1    1        7        7          120     20        10  4   
8         9    1    1        7        7          120     20        10  4   
9        10    1    1        7        7          120     20        10  4   
10       11    1    1        7        7          120     20        10  4   
11       12    1    1        7        7          120 

In [118]:
import pandas as pd
import numpy as np

# 1) Cargo la base histórica y calculo min/max
df_base = pd.read_csv('datos_resumen.csv').rename(columns={
    'Nexpl': 'nexpl',
    'Nexploit': 'nexpt',
    'dist_colony': 'dist_col'
})

# Columnas que intervienen en eficiencia
output_cols = ['visitas_con', 'nectar', 'visitas_sin', 't_refill_med']
min_vals = df_base[output_cols].min()
max_vals = df_base[output_cols].max()

# Pesos originales
w1, w2, w3, w4 = 0.3, 0.3, 0.2, 0.2

# 2) Función de eficiencia (idéntica a la que usas en summary)
def calcular_eficiencia_con_tfill(fila):
    c1 = (fila['visitas_con_nectar']       - min_vals['visitas_con'])    / (max_vals['visitas_con']    - min_vals['visitas_con'])
    c2 = (fila['total_nectar_recolectado']  - min_vals['nectar'])         / (max_vals['nectar']         - min_vals['nectar'])
    c3 = (fila['visitas_sin_nectar']       - min_vals['visitas_sin'])    / (max_vals['visitas_sin']    - min_vals['visitas_sin'])
    c4 = (fila['t_medio_reposicion']       - min_vals['t_refill_med'])   / (max_vals['t_refill_med']   - min_vals['t_refill_med'])

    E_raw = w1*c1 + w2*c2 - w3*c3 - w4*c4

    # calculo eficiencia “raw” para la base
    norm_base = (
        w1 * (df_base['visitas_con']   - min_vals['visitas_con'])    / (max_vals['visitas_con']    - min_vals['visitas_con'])
      + w2 * (df_base['nectar']        - min_vals['nectar'])         / (max_vals['nectar']         - min_vals['nectar'])
      - w3 * (df_base['visitas_sin']   - min_vals['visitas_sin'])    / (max_vals['visitas_sin']    - min_vals['visitas_sin'])
      - w4 * (df_base['t_refill_med']  - min_vals['t_refill_med'])   / (max_vals['t_refill_med']   - min_vals['t_refill_med'])
    )
    E_min, E_max = norm_base.min(), norm_base.max()

    return (E_raw - E_min) / (E_max - E_min)

# 3) Define aquí tu nuevo vector (por ejemplo, el que dio el MLP)
nuevo = {
    'visitas_con_nectar':       257.48,
    'visitas_sin_nectar':       335.80,
    'total_nectar_recolectado': 273.46,
    't_medio_reposicion':       4.71
}

# 4) Calculo eficiencia
df_nuevo = pd.DataFrame([nuevo])
ef = calcular_eficiencia_con_tfill(df_nuevo.iloc[0])
print(f"Eficiencia normalizada para el vector dado: {ef:.4f}")


Eficiencia normalizada para el vector dado: 0.6477
